In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [3]:
train_df="/kaggle/input/surface-crack/train"
test_df="/kaggle/input/surface-crack/test"
valid_df="/kaggle/input/surface-crack/valid"

In [4]:
train_data = train_gen.flow_from_directory(
    train_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,

 
)

val_data = train_gen.flow_from_directory(
    valid_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True

   

)

test_data = train_gen.flow_from_directory(
    test_df,
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,

    
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,RMSprop, Adagrad

from keras.layers import BatchNormalization

In [6]:
 def random_invert_img(x, p=0.6):
    if tf.random.uniform([]) < p:
        x = (255 - x)
    else:
        x
    return x
 def random_inv(factor = 0.65):
    return tf.keras.layers.Lambda(lambda x:random_invert_img(x, factor)) 

In [7]:
model = Sequential()

model.add(tf.keras.layers.RandomContrast(0.2,0.7))
model.add(tf.keras.layers.RandomCrop(120,120))
model.add(tf.keras.layers.RandomRotation(0.2))
model.add(tf.keras.layers.RandomFlip("horizontal_and_vertical"))
model.add(tf.keras.layers.RandomContrast(0.2,0.7))
model.add(tf.keras.layers.RandomHeight(0.3))
model.add(random_inv())

model.add(Conv2D(32,3,padding="same", activation="relu", input_shape = (120,120,3)))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(128, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.5))

model.add(BatchNormalization())
model.add(Dense(1, activation="sigmoid"))


model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

2023-01-14 12:34:11.867350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 12:34:11.871240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 12:34:11.872005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 12:34:11.874191: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x=tf.keras.layers.RandomContrast(0.2,0.7)
x=tf.keras.layers.RandomCrop(120,120)
x=tf.keras.layers.RandomRotation(0.2)
x=tf.keras.layers.RandomFlip("horizontal_and_vertical")
x=tf.keras.layers.RandomHeight(0.3)
x=random_inv()





x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 118, 118, 32)      896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 57, 57, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 128)       0     

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [10]:
filepath="we111ights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
callbacks_list = [checkpoint, es]

In [11]:


history = model.fit(train_data,validation_data=val_data,epochs = 100,callbacks=callbacks_list ,verbose=1)

2023-01-14 12:34:15.222427: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-14 12:34:17.168682: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


19/19 [==============================] - 12s 229ms/step - loss: 0.6106 - accuracy: 0.6500 - val_loss: 0.6566 - val_accuracy: 0.6350

Epoch 00001: val_loss improved from inf to 0.65659, saving model to we111ights.best.hdf5
Epoch 2/100
19/19 [==============================] - 2s 89ms/step - loss: 0.5182 - accuracy: 0.7483 - val_loss: 0.6225 - val_accuracy: 0.7200

Epoch 00002: val_loss improved from 0.65659 to 0.62254, saving model to we111ights.best.hdf5
Epoch 3/100
19/19 [==============================] - 2s 85ms/step - loss: 0.4511 - accuracy: 0.7767 - val_loss: 0.6494 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 0.62254
Epoch 4/100
19/19 [==============================] - 2s 98ms/step - loss: 0.4245 - accuracy: 0.8100 - val_loss: 0.6264 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 0.62254
Epoch 5/100
19/19 [==============================] - 2s 88ms/step - loss: 0.3786 - accuracy: 0.8500 - val_loss: 0.6880 - val_accuracy: 0.5000

Epoch 00005

In [12]:
fig = px.line( 
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
)

fig.show()

In [13]:
def evaluate_model(model, val_data):
    
    results = model.evaluate(val_data, verbose=0)
    loss = results[0]
    acc = results[1]
    print("    Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))
    y_pred = np.squeeze((model.predict(test_data) >= 0.5).astype(np.int))
   
    clr = classification_report(test_data.labels, y_pred, target_names=["NEGATIVE", "POSITIVE"])
    print("Classification Report:\n----------------------\n", clr)
                                                                        



**classifcation report for Test data**

In [18]:
evaluate_model(model, test_data)

    Test Loss: 0.26199
Test Accuracy: 91.50%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.85      1.00      0.92       100
    POSITIVE       1.00      0.83      0.91       100

    accuracy                           0.92       200
   macro avg       0.93      0.92      0.91       200
weighted avg       0.93      0.92      0.91       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



**classifcation report for validation data**

In [15]:
evaluate_model(model, val_data)

    Test Loss: 0.14850
Test Accuracy: 94.50%
Classification Report:
----------------------
               precision    recall  f1-score   support

    NEGATIVE       0.85      1.00      0.92       100
    POSITIVE       1.00      0.83      0.91       100

    accuracy                           0.92       200
   macro avg       0.93      0.92      0.91       200
weighted avg       0.93      0.92      0.91       200



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [16]:
y_pred.size

NameError: name 'y_pred' is not defined